In [1]:
import pandas as pd
import numpy as np

### Описание данных

**courses.csv** — файл содержит список предметов по семестрам.  

```code_module``` — предмет (идентификационный код).

```code_presentation``` — семестр (идентификационный код).

```module_presentation_length``` — продолжительность семестра в днях.

**assessments.csv** — этот файл содержит информацию об оценках в тесте. Обычно каждый предмет в семестре включает ряд тестов с оценками, за которыми следует заключительный экзаменационный тест (экзамен).  

```code_module``` — идентификационный код предмета.  

```code_presentation``` — семестр (Идентификационный код).  

```id_assessment``` — тест (Идентификационный номер ассессмента).  

```assessment_type``` — тип теста. Существуют три типа оценивания: оценка преподавателя (TMA), компьютерная оценка (СМА), экзамен по курсу (Exam).  

```date``` — информация об окончательной дате сдачи теста. Рассчитывается как количество дней с момента начала семестра. Дата начала семестра имеет номер 0 (ноль).  

```weight``` — вес теста в % в оценке за курс. Обычно экзамены рассматриваются отдельно и имеют вес 100%; сумма всех остальных оценок составляет 100%.  

**studentAssessment.csv** — этот файл содержит результаты тестов студентов. Если учащийся не отправляет работу на оценку, результат не записывается в таблицу.  

```id_assessment``` — тест (идентификационный номер).  

```id_student``` — идентификационный номер студента.  

```date_submitted``` — дата сдачи теста студентом, измеряемая как количество дней с начала семестра.  

```is_banked``` — факт перезачета теста с прошлого семестра (иногда курсы перезачитывают студентам, вернувшимся из академического отпуска).  

```score``` — оценка учащегося в этом тесте. Диапазон составляет от 0 до 100. Оценка ниже 40 неудачная/неуспешная сдача теста.

**studentRegistration.csv** — этот файл содержит информацию о времени, когда студент зарегистрировался для прохождения курса в семестре.  

```code_module``` — предмет (идентификационный код).  

```code_presentation``` — семестр (идентификационный код)  

```id_student``` — идентификационный номер студента.  

```date_registration``` — дата регистрации студента. Это количество дней, измеренное от начала семестра (например, отрицательное значение -30 означает, что студент зарегистрировался на прохождение курса за 30 дней до его начала).  

```date_unregistration``` — дата отмены регистрации студента с предмета. У студентов, окончивших курс, это поле остается пустым.

In [2]:
assessments = pd.read_csv('assessments.csv')
courses = pd.read_csv('courses.csv')
studentAssessment = pd.read_csv('studentAssessment.csv')
studentRegistration = pd.read_csv('studentRegistration.csv')

Для решения задачи проведи предварительное исследование данных и сформулируй, что должно считаться курсом. Обосновать свой выбор ты можешь с помощью фактов сдачи экзаменов, распределения студентов и уникального идентификатора курса.

Продакт-менеджер Василий попросил вас проанализировать завершенные уроки и ответить на следующие вопросы:  

1. Сколько студентов успешно сдали только один курс? (Успешная сдача — это зачёт по курсу на экзамене) (7 баллов).  

2. Выяви самый сложный и самый простой экзамен: найди курсы и экзамены в рамках курса, которые обладают самой низкой и самой высокой завершаемостью*. (10 баллов)  

3. По каждому предмету определи средний срок сдачи экзаменов (под сдачей понимаем последнее успешное прохождение экзамена студентом). (7 баллов)   

4. Выяви самые популярные предметы (ТОП-3) по количеству регистраций на них. А также предметы с самым большим оттоком (ТОП-3). (8 баллов)  

5. Используя pandas, в период с начала 2013 по конец 2014 выяви семестр с самой низкой завершаемостью курсов и самыми долгими средними сроками сдачи курсов.  (15 баллов)   

6. Часто для качественного анализа аудитории используют подходы, основанные на сегментации. Используя python, построй адаптированные RFM-кластеры студентов, чтобы качественно оценить свою аудиторию. В адаптированной кластеризации можешь выбрать следующие метрики: R - среднее время сдачи одного экзамена, F - завершаемость курсов, M - среднее количество баллов, получаемое за экзамен. Подробно опиши, как ты создавал кластеры. Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров. (23 балла)  

*завершаемость = кол-во успешных экзаменов / кол-во всех попыток сдать экзамен

Посмотрим на данные

In [3]:
courses.shape

(22, 3)

In [4]:
courses.head()

,code_module,code_presentation,module_presentation_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268
3,BBB,2014J,262
4,BBB,2013B,240


In [5]:
courses.describe()

,module_presentation_length
count,22.000000
mean,255.545455
std,13.654677
min,234.000000
25%,241.000000
50%,261.500000
75%,268.000000
max,269.000000


In [6]:
assessments.shape

(206, 6)

In [7]:
assessments.head()

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0


In [8]:
assessments.describe()

,id_assessment,date,weight
count,206.000000,195.000000,206.000000
mean,26473.975728,145.005128,20.873786
std,10098.625521,76.001119,30.384224
min,1752.000000,12.000000,0.000000
25%,15023.250000,71.000000,0.000000
50%,25364.500000,152.000000,12.500000
75%,34891.750000,222.000000,24.250000
max,40088.000000,261.000000,100.000000


In [9]:
studentAssessment.shape

(173912, 5)

In [10]:
studentAssessment.head()

,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78.0
1,1752,28400,22,0,70.0
2,1752,31604,17,0,72.0
3,1752,32885,26,0,69.0
4,1752,38053,19,0,79.0


In [11]:
studentAssessment.describe()

,id_assessment,id_student,date_submitted,is_banked,score
count,173912.000000,1.739120e+05,173912.000000,173912.000000,173739.000000
mean,26553.803556,7.051507e+05,116.032942,0.010977,75.799573
std,8829.784254,5.523952e+05,71.484148,0.104194,18.798107
min,1752.000000,6.516000e+03,-11.000000,0.000000,0.000000
25%,15022.000000,5.044290e+05,51.000000,0.000000,65.000000
50%,25359.000000,5.852080e+05,116.000000,0.000000,80.000000
75%,34883.000000,6.344980e+05,173.000000,0.000000,90.000000
max,37443.000000,2.698588e+06,608.000000,1.000000,100.000000


In [12]:
studentRegistration.shape

(32593, 5)

In [13]:
studentRegistration.head()

,code_module,code_presentation,id_student,date_registration,date_unregistration
0,AAA,2013J,11391,-159.0,NaN
1,AAA,2013J,28400,-53.0,NaN
2,AAA,2013J,30268,-92.0,12.0
3,AAA,2013J,31604,-52.0,NaN
4,AAA,2013J,32885,-176.0,NaN


In [14]:
studentRegistration.describe()

,id_student,date_registration,date_unregistration
count,3.259300e+04,32548.000000,10072.000000
mean,7.066877e+05,-69.411300,49.757645
std,5.491673e+05,49.260522,82.460890
min,3.733000e+03,-322.000000,-365.000000
25%,5.085730e+05,-100.000000,-2.000000
50%,5.903100e+05,-57.000000,27.000000
75%,6.444530e+05,-29.000000,109.000000
max,2.716795e+06,167.000000,444.000000


### Задание 0
Что можно считать курсом?
Курс = связка ```code_module``` и ```code_presentation```. То есть это **предмет** (```code_module```), который студент  берёт в **семестре** (```code_presentation```). 

Дополнительно посчитаем, сколько всего у нас уникальных студентов (для проверок).

In [70]:
uniqueStudentRegistrations = set(studentRegistration.id_student.unique())

In [71]:
uniqueStudentAssessments = set(studentAssessment.id_student.unique())

In [82]:
len(uniqueStudentRegistrations)

28785

In [83]:
len(uniqueStudentAssessments)

23369

In [79]:
uniqueStudentsDifferences = uniqueStudentRegistrations - uniqueStudentAssessments
len(uniqueStudentsDifferences)

5416

5416 студентов, которые регистрировались, но не сдавали тесты? 

### Задание 1
Сколько студентов успешно сдали только один курс? (Успешная сдача — это зачёт по курсу на экзамене) (7 баллов). 

In [139]:
assessments_total = pd.merge(studentAssessment, assessments, on='id_assessment', how='left')

In [140]:
assessments_total.shape

(173912, 10)

In [141]:
assessments_total['success_exam'] = assessments_total.score >= 40

In [142]:
assessments_total.id_student.nunique()

23369

In [143]:
assessments_total.query('weight == 100 & assessment_type == "Exam"') \
                .groupby('id_student', as_index=False) \
                .agg({'success_exam' : 'count'}) \
                .query('success_exam == 1').shape

(4307, 2)

Количество студентов, успешно сдавших только один курс – **4307**   

### Задание 2

Выяви самый сложный и самый простой экзамен: найди курсы и экзамены в рамках курса, которые обладают самой низкой и самой высокой завершаемостью. Завершаемость = кол-во успешных экзаменов / кол-во всех попыток сдать экзамен

In [156]:
assessments_total.head()

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight,success_exam
0,1752,11391,18,0,78.0,AAA,2013J,TMA,19.0,10.0,True
1,1752,28400,22,0,70.0,AAA,2013J,TMA,19.0,10.0,True
2,1752,31604,17,0,72.0,AAA,2013J,TMA,19.0,10.0,True
3,1752,32885,26,0,69.0,AAA,2013J,TMA,19.0,10.0,True
4,1752,38053,19,0,79.0,AAA,2013J,TMA,19.0,10.0,True


In [157]:
total_attempts = assessments_total.query('assessment_type == "Exam"') \
    .groupby(['id_assessment', 'assessment_type', 'code_module', 'code_presentation']).agg(
        total_attempts=('success_exam', 'count'),  
        successful_attempts=('success_exam', 'sum')  
    ).reset_index()

In [158]:
total_attempts

,id_assessment,assessment_type,code_module,code_presentation,total_attempts,successful_attempts
0,24290,Exam,CCC,2014B,747,664
1,24299,Exam,CCC,2014J,1168,1019
2,25340,Exam,DDD,2013B,602,504
3,25354,Exam,DDD,2013J,968,878
4,25361,Exam,DDD,2014B,524,485
5,25368,Exam,DDD,2014J,950,842


In [159]:
total_attempts['total'] = total_attempts.successful_attempts / total_attempts.total_attempts

In [160]:
total_attempts.sort_values(by='total')

,id_assessment,assessment_type,code_module,code_presentation,total_attempts,successful_attempts,total
2,25340,Exam,DDD,2013B,602,504,0.837209
1,24299,Exam,CCC,2014J,1168,1019,0.872432
5,25368,Exam,DDD,2014J,950,842,0.886316
0,24290,Exam,CCC,2014B,747,664,0.888889
3,25354,Exam,DDD,2013J,968,878,0.907025
4,25361,Exam,DDD,2014B,524,485,0.925573


Самый низкий – **DDD 2013B**,  самый высокий – **DDD 2014B**

### Задание 3

По каждому предмету определи средний срок сдачи экзаменов (под сдачей понимаем последнее успешное прохождение экзамена студентом). (7 баллов) 

In [161]:
assessments_total.head()

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight,success_exam
0,1752,11391,18,0,78.0,AAA,2013J,TMA,19.0,10.0,True
1,1752,28400,22,0,70.0,AAA,2013J,TMA,19.0,10.0,True
2,1752,31604,17,0,72.0,AAA,2013J,TMA,19.0,10.0,True
3,1752,32885,26,0,69.0,AAA,2013J,TMA,19.0,10.0,True
4,1752,38053,19,0,79.0,AAA,2013J,TMA,19.0,10.0,True


In [185]:
assessments_total.query('weight == 100 & score >= 40 & assessment_type == "Exam"') \
                .groupby('code_module', as_index=False) \
                .agg({'date_submitted':'mean'}) \
                .round(2)

,code_module,date_submitted
0,CCC,239.35
1,DDD,237.98


Средний срок сдачи экзамена по предмету ССС: **239,35**. По предмету DDD: **237,98**

Если же имеем в виду курсы (code_module + code_presentation), получим следующие средние значения:

In [186]:
assessments_total.query('weight == 100 & score >= 40 & assessment_type == "Exam"') \
                .groupby(['code_module', 'code_presentation'], as_index=False) \
                .agg({'date_submitted':'mean'}) \
                .round(2)

,code_module,code_presentation,date_submitted
0,CCC,2014B,231.58
1,CCC,2014J,244.40
2,DDD,2013B,230.16
3,DDD,2013J,239.51
4,DDD,2014B,234.94
5,DDD,2014J,242.80


### Задание 4

Выяви самые популярные предметы (ТОП-3) по количеству регистраций на них. А также предметы с самым большим оттоком (ТОП-3). (8 баллов)

1. Количество регистраций

In [196]:
studentRegistration.query('date_registration != "NaN"') \
                    .groupby('code_module', as_index=False) \
                    .agg({'id_student' : 'nunique'}) \
                    .sort_values(by='id_student', ascending=False).head(3)

,code_module,id_student
1,BBB,7683
5,FFF,7387
3,DDD,5840


Предметы с самым большим количеством регистраций: BBB, FFF, DDD.  

2. Отток

In [197]:
studentRegistration.query('date_unregistration != "NaN"') \
                    .groupby('code_module', as_index=False) \
                    .agg({'id_student' : 'nunique'}) \
                    .sort_values(by='id_student', ascending=False).head(3)

,code_module,id_student
1,BBB,2314
5,FFF,2249
3,DDD,2065


### Задание 5

Используя pandas, в период с начала 2013 по конец 2014 выяви семестр с самой низкой завершаемостью курсов и самыми долгими средними сроками сдачи курсов.  (15 баллов) 